# 6. Tuning

In [1]:
MODEL = 'RFR'
TUNING_MODEL = 'jiaocheng'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import JiaoCheng_H2H

In [11]:
jiaocheng = JiaoCheng_H2H.JiaoCheng()

JiaoCheng Initialised


In [12]:
from sklearn.ensemble import RandomForestRegressor as RFR

In [13]:
jiaocheng.read_in_model(RFR, 'Regression')

Successfully read in model <class 'sklearn.ensemble._forest.RandomForestRegressor'>, which is a Regression model


In [14]:
jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [15]:
parameter_choices = {
    'n_estimators': (25, 50, 100, 200, 400),
    'max_depth': (3, 6, 12, 24, 48, 96, 192),
    'max_samples': (0.4, 0.55, 0.7, 0.85),
    'max_features': (0.4, 0.55, 0.7, 0.85),
    'ccp_alpha': (0.00000001, 0.000001, 0.0001, 0.01, 0.1)
}

In [16]:
jiaocheng.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [17]:
jiaocheng.set_non_tuneable_hyperparameters({'random_state': 19260817, 'n_jobs': -1})

Successfully recorded non_tuneable_hyperparameter choices


In [18]:
jiaocheng.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [19]:
jiaocheng.set_tuning_order(['features', 'max_depth', 'max_features', 'max_samples', 'n_estimators', 'ccp_alpha'])

In [20]:
jiaocheng.set_hyperparameter_default_values({
    'features': 0,
    'n_estimators': 50,
    'max_depth': 12,
    'max_samples': 0.7,
    'max_features': 0.7,
    'ccp_alpha': 0.01})

In [21]:
try:
  jiaocheng.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')
  print('True')
except:
  print('False')

Successfully read in tuning result of 156 rows
True


In [22]:
jiaocheng.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [23]:
jiaocheng.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [24]:
jiaocheng.train_other_info = train_other_info
jiaocheng.val_other_info = val_other_info
jiaocheng.test_other_info = test_other_info

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
jiaocheng.tune()


Default combo: [1, 2, 2, 2, 3, 0] 


ROUND 1

Round 2 
Hyperparameter: features (index: 5) 

As new Best Combo (1, 2, 2, 2, 3, 0) was read in, best_clf is set to None
Already Trained and Tested combination (1, 2, 2, 2, 3, 0), which had val score of 0.5954
        Current best combo: (1, 2, 2, 2, 3, 0) with val score 0.5954. 
        Has trained 156 of 100800 combinations so far
Already Trained and Tested combination (1, 2, 2, 2, 3, 1), which had val score of 0.5922
        Current best combo: (1, 2, 2, 2, 3, 0) with val score 0.5954. 
        Has trained 156 of 100800 combinations so far
As new Best Combo (1, 2, 2, 2, 3, 2) was read in, best_clf is set to None
Already Trained and Tested combination (1, 2, 2, 2, 3, 2), which had val score of 0.6027
        Current best combo: (1, 2, 2, 2, 3, 2) with val score 0.6027. 
        Has trained 156 of 100800 combinations so far
Already Trained and Tested combination (1, 2, 2, 2, 3, 3), which had val score of 0.5713
        Current best combo:

In [27]:
jiaocheng.view_best_combo_and_score()

Max Score: 
 0.607966
Max Test Score: 
 0.611111
Max Combo Index: 
 (0, 1, 2, 2, 3, 4) out of [5, 7, 4, 4, 5, 36] (note best combo is 0-indexed)
Max Combo Hyperparamer Combination: 
 {'n_estimators': 25, 'max_depth': 6, 'max_samples': 0.7, 'max_features': 0.7, 'ccp_alpha': 0.01, 'features': 4}
Max Combo Features: 
 ('AFLCA_votes', 'Disposals', 'Contested Possessions', 'Goals', 'Centre Clearances', 'Intercepts', 'Clearances')
% Combos Checked: 168 out of 100800 which is 0.166667%
